# Advanced SQL

## Functions and Procedures

We have already seen several functions that are built into the SQL language. In this section, we show how developers can write their ow functions and procedures, and **store them in the database** so as to invoke them from SQL statements. 

SQL allows for the definition of functions, procedures, and methods.

### Declaring and Invoking MSSQL functions and procedures

Suppose that we want a function that, given the name of a department, returns the count of the number of instructors in that department. 

In [6]:
-- select the database
USE uni;
GO
-- existing, drop it
IF EXISTS (SELECT  * FROM dbo.sysobjects WHERE id = OBJECT_ID(N'[dbo].[instructor_count]'))
    BEGIN
        DROP FUNCTION dbo.instructor_count
    END;
GO

-- define the function
CREATE FUNCTION dbo.instructor_count(@dept_name VARCHAR(20))
RETURNS INT 
AS
BEGIN
    -- Declare the return variable here
    DECLARE @instructor_n INT
    -- Make the query
    SELECT @instructor_n = COUNT(*)
    FROM instructor
    WHERE instructor.dept_name = @dept_name
    -- return
    RETURN @instructor_n
END;
GO


Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.017

In [4]:
-- test it
SELECT dbo.instructor_count('Statistics') AS inst_n;
GO

SELECT COUNT(*) AS inst_n
FROM instructor
WHERE instructor.dept_name = 'Statistics';
GO

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.036

inst_n
6


inst_n
6


This function returns all the professors names as well as ids for a department. This is an example of a table function which is relatively different in its definition from scalar output functions such as the one above.

In [7]:
-- existing, drop it
IF EXISTS (SELECT  * FROM dbo.sysobjects WHERE id = OBJECT_ID(N'[dbo].[dept_instructors]'))
    BEGIN
        DROP FUNCTION dbo.dept_instructors
    END;
GO

CREATE FUNCTION dbo.dept_instructors(@dept_name VARCHAR(20))
RETURNS TABLE
AS 
RETURN(
    SELECT [name], ID 
    FROM instructor 
    WHERE dept_name = @dept_name 
);
GO


Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.012

In [8]:
-- test it
SELECT * FROM dbo.dept_instructors('Statistics');
GO

SELECT [name], ID 
FROM instructor 
WHERE dept_name = 'Statistics';
GO

(6 rows affected)

(6 rows affected)

Total execution time: 00:00:00.018

name,ID
Atanassov,28400
Arias,37687
Gutierrez,64871
Pingr,78699
Choll,90643
Arinb,95030


name,ID
Atanassov,28400
Arias,37687
Gutierrez,64871
Pingr,78699
Choll,90643
Arinb,95030


Below we generate a stored procedure which does exactly the same.

In [9]:
-- existing, drop it
IF EXISTS (SELECT  * FROM dbo.sysobjects WHERE id = OBJECT_ID(N'[dbo].[dept_instructors_proc]'))
    BEGIN
        DROP PROCEDURE dbo.dept_instructors_proc
    END;
GO

-- define the stored procedure
CREATE PROCEDURE dbo.dept_instructors_proc(@dept_name VARCHAR(20))
AS
BEGIN
    SELECT [name], ID 
    FROM instructor 
    WHERE dept_name = @dept_name
END;
GO

-- Running it
EXEC dept_instructors_proc @dept_name = 'Statistics';
GO

Commands completed successfully.

Commands completed successfully.

(6 rows affected)

Total execution time: 00:00:00.018

name,ID
Atanassov,28400
Arias,37687
Gutierrez,64871
Pingr,78699
Choll,90643
Arinb,95030


### Stored procedures versus functions

While the output is the same, there are differences beyound just how they are coded:

- **Functions are computed values and cannot perform permanent changes to the server, whereas stored procedures can**
- **[The output of] functions can be using within SQL query, stored procedures, since they need the ``EXEC`` statement to be ran, cannot.**
- **Stored procedures can return zero, single or multiple values; functions must return a single value, being either a scalar or a table**
- **We can use exception handling using Try-Catch blocks in stored procedures, but we cannot within functions**
- **We can define input paramters for functions; we can define both input as well as output parameters for stored procedures**

Found this [thread](https://stackoverflow.com/questions/1179758/function-vs-stored-procedure-in-sql-server) really helpful.

In [16]:
-- a stored procedure with an output parameter
-- existing, drop it
IF EXISTS (SELECT  * FROM dbo.sysobjects WHERE id = OBJECT_ID(N'[dbo].[dept_instructors_proc2]'))
    BEGIN
        DROP PROCEDURE dbo.dept_instructors_proc2
    END;
GO

-- define the stored procedure
CREATE PROCEDURE dbo.dept_instructors_proc2 (
    @dept_name VARCHAR(20),
    @instructor_count INT OUTPUT  
) AS
BEGIN
    SELECT [name], ID 
    FROM instructor 
    WHERE dept_name = @dept_name;

    -- prepare the output
    SELECT @instructor_count = @@ROWCOUNT;
END;
GO

-- declare the output parameter
DECLARE @count INT
-- Running it
EXEC dept_instructors_proc2 @dept_name = 'Statistics', @instructor_count = @count OUTPUT
-- retrieve the output
SELECT @count AS 'Number of instructors found';
GO

Commands completed successfully.

Commands completed successfully.

(6 rows affected)

(1 row affected)

Total execution time: 00:00:00.014

name,ID
Atanassov,28400
Arias,37687
Gutierrez,64871
Pingr,78699
Choll,90643
Arinb,95030


Number of instructors found
6


## Control-of-Flow statements

One of the nice things of functions and stored procedures is that we can make use of several control of flow statements.

### The `BEGIN...END` statement

The BEGIN...END statement is used to define a statement block. A statement block consists of a set of SQL statements that execute together. A statement block is also known as a batch.

```
BEGIN
    { sql_statement | statement_block}
END
```

For example, in the code belo we combine a sql query and printing a message conditional on the number of rows in the output table of that query. We can do this because we included them in a batch. Note that ``@@ROWCOUNT`` is a system variable that returns the number of rows affected by the last previous statement.

In [17]:
BEGIN
    SELECT [name], ID 
    FROM instructor 
    WHERE dept_name = 'Statistics';

IF @@ROWCOUNT > 5
    PRINT 'There are more than 5 instructors in the department'
ELSE
    PRINT 'There are less than 5 instructors in the department'
END;
GO

(6 rows affected)

There are more than 5 instructors in the department

Total execution time: 00:00:00.003

name,ID
Atanassov,28400
Arias,37687
Gutierrez,64871
Pingr,78699
Choll,90643
Arinb,95030


We can also nest several begin end statements. For example

In [23]:
BEGIN 
    -- declare the variables we will use in this batch
    DECLARE @student_name VARCHAR(40)
    DECLARE @dept_name VARCHAR(40)
    DECLARE @tc INT

    -- the query: highest total credits
    SELECT TOP 1
        @student_name = [name],
        @dept_name = dept_name,
        @tc = tot_cred
    FROM student
    ORDER BY tot_cred DESC;

    -- second level
    IF @dept_name IN ('Biology', 'Comp. Sci.', 'Statistics', 'Math', 'Mech Eng.', 'Civil Eng.')
        BEGIN
            PRINT 'The student with the highest total credits is ' + @student_name  +' (STEM FIELD) with ' + CONVERT(varchar(10), @tc)
        END
    ELSE
        BEGIN
            PRINT 'The student with the highest total credits is ' + @student_name + ' (NON-STEM FIELD) with ' + CONVERT(varchar(10), @tc)
        END;
END;

The student with the highest total credits is Yoneda (NON-STEM FIELD) with 129

Total execution time: 00:00:00.004

### ``IF...ELSE``

When the condition in the IF clause evaluates to FALSE and you want to execute another statement block, you can use the ELSE clause.

```
IF Boolean_expression
BEGIN
    -- Statement block executes when the Boolean expression is TRUE
END
ELSE
BEGIN
    -- Statement block executes when the Boolean expression is FALSE
END
```
For example, say that we want to check whether more than half of the students in a certain class failed

In [39]:
BEGIN
    -- Declare the variables to be used in the batch
    DECLARE @rel_dept AS VARCHAR(20)
    SET @rel_dept = 'Statistics'
    DECLARE @percent_fail AS FLOAT
    -- the query
    SET @percent_fail = (
        SELECT AVG(CAST(IIF(takes.grade LIKE 'C_', 1, 0) AS FLOAT))
        FROM takes, student, course
        WHERE 
            takes.ID = student.ID AND 
            course.course_id = takes.course_id AND 
            course.dept_name = @rel_dept
    );
    -- messages
    IF (@percent_fail > 0.5)
        BEGIN
            PRINT 'More than half of students failed, namely ' + CONVERT(VARCHAR(10), ROUND(@percent_fail * 100, 2)) + '%'
        END
    ELSE
        BEGIN
            PRINT 'Less than half of students failed, namely ' + CONVERT(VARCHAR(10), ROUND(@percent_fail * 100, 2)) + '%'
        END;
END;


Less than half of students failed, namely 35.31%

Total execution time: 00:00:00.057

Note that we can nest them too and use ``ELSE IF``.

In [47]:
BEGIN
    -- Declare the variables to be used in the batch
    DECLARE @rel_dept AS VARCHAR(20)
    SET @rel_dept = 'Biology'
    DECLARE @percent_fail AS FLOAT
    -- the query
    SET @percent_fail = (
        SELECT AVG(CAST(IIF(takes.grade LIKE 'C_', 1, 0) AS FLOAT))
        FROM takes, student, course
        WHERE 
            takes.ID = student.ID AND 
            course.course_id = takes.course_id AND 
            course.dept_name = @rel_dept
    );
    -- messages
    IF (@percent_fail > 0.5) AND @rel_dept IN ('Biology', 'Comp. Sci.', 'Statistics', 'Math', 'Mech Eng.', 'Civil Eng.')
        BEGIN
            PRINT 'More than half of students failed in this STEM-field, namely ' + CONVERT(VARCHAR(10), ROUND(@percent_fail * 100, 2)) + '%'
        END
    ELSE IF (@percent_fail > 0.5) AND @rel_dept IN ('Biology', 'Comp. Sci.', 'Statistics', 'Math', 'Mech Eng.', 'Civil Eng.')
        BEGIN
            PRINT 'More than half of students failed in this STEM-field, namely ' + CONVERT(VARCHAR(10), ROUND(@percent_fail * 100, 2)) + '%'
        END
    ELSE IF (@percent_fail < 0.5) AND @rel_dept IN ('Biology', 'Comp. Sci.', 'Statistics', 'Math', 'Mech Eng.', 'Civil Eng.')
        BEGIN
            PRINT 'More than half of students failed in this STEM-field, namely ' + CONVERT(VARCHAR(10), ROUND(@percent_fail * 100, 2)) + '%'
        END
    ELSE
        BEGIN
            PRINT 'Less than half of students failed, namely ' + CONVERT(VARCHAR(10), ROUND(@percent_fail * 100, 2)) + '%'
        END;
END;

More than half of students failed in this STEM-field, namely 33.96%

Total execution time: 00:00:00.032

### `WHILE`

The WHILE statement is a control-flow statement that allows you to execute a statement block repeatedly as long as a specified condition is TRUE.

First, the Boolean_expression is an expression that evaluates to TRUE or FALSE.

Second, <code>sql_statement | statement_block</code> is any Transact-SQL statement or a set of Transact-SQL statements. A statement block is defined using the <code>BEGIN...END</code> statement.

If the Boolean_expression evaluates to FALSE in an iteration, no statement inside the WHILE loop will be executed.

Inside the WHILE loop, you must change some variables to make the Boolean_expression returns FALSE at some points. Otherwise, you will have an indefinite loop.

Furthermore:

- `BREAK` - Causes an exit from the innermost WHILE loop. Any statements that appear after the END keyword, marking the end of the loop, are executed
- `CONTINUE` - Causes the WHILE loop to restart, ignoring any statements after the `CONTINUE` keyword, i.e. skiping the current itearation

```
WHILE Boolean_expression
{ sql_statement | statement_block | BREAK | CONTINUE }

```

Below an example.Double the budget of departments with below average budgets until the average budget is higher than 3,000,000. However, never allow one of them to be above 3,000,000 after the doubling.

In [45]:
-- before
SELECT AVG(budget) FROM department;
GO

-- Start the while loop
WHILE (SELECT AVG(budget) FROM department) < 3000000
BEGIN
    -- declare the relevant variables
    DECLARE @cur_depts AS VARCHAR(20)
    -- Below average departments
    SELECT @cur_depts = dept_name
    FROM department
    WHERE budget < (SELECT AVG(budget) FROM department);
    -- alter the database
    UPDATE department
    SET budget = budget * 2;
    -- message
    PRINT 'Doubling the budget from ' + COALESCE(@cur_depts, ', ', '')
    -- no budget can exceed 3,000,000
    IF (SELECT MAX(budget) FROM department) > 3000000
        BREAK
    ELSE
        CONTINUE
END;
GO

-- after
SELECT AVG(budget) FROM department;
GO


(1 row affected)

(20 rows affected)

Doubling the budget from Statistics

(1 row affected)

Total execution time: 00:00:00.013

(No column name)
2265264.020000


(No column name)
4530528.040000


## Database cursor

A database cursor is an object that enables traversal over the rows of a result set. It allows you to process individual row returned by a query.

It involves the following steps.

1. Declare the relevant variables to hold the values of the rows we are after

In [56]:
DECLARE @student_id INT;

Commands completed successfully.

Total execution time: 00:00:00.001

2. Declare the cursor

In [57]:
DECLARE below_totavg_cursor CURSOR
    FOR SELECT
        ID
        FROM student
        WHERE tot_cred < (SELECT AVG(tot_cred) FROM student);

Commands completed successfully.

Total execution time: 00:00:00.001

3. Next, open the cursor

In [58]:
OPEN below_totavg_cursor;

Commands completed successfully.

Total execution time: 00:00:00.006

4. Then, fetch each row from the cursor and print out the product name and list price.

In [59]:
WHILE @@FETCH_STATUS = 0
    BEGIN 
        PRINT 'Student with the ID ' + @student_id + 'has below average total credits'
        FETCH NEXT FROM below_totavg_cursor INTO
            @student_id
    END;
GO

: Msg 137, Level 15, State 2, Line 3
Must declare the scalar variable "@student_id".

: Msg 137, Level 15, State 2, Line 5
Must declare the scalar variable "@student_id".

Total execution time: 00:00:00.001

5. Close and deallocate the cursor

In [55]:
CLOSE below_totavg_cursor;
DEALLOCATE below_totavg_cursor;

Commands completed successfully.

Total execution time: 00:00:00

All in one with 110 credits has a treshhold (less printing).

In [1]:
USE uni;
GO

DECLARE @student_id VARCHAR(20);

DECLARE above_110_cursor CURSOR
    FOR SELECT
        [ID]
        FROM student
        WHERE tot_cred > 110;

OPEN above_110_cursor;

WHILE @@FETCH_STATUS = 0
    BEGIN 
        PRINT 'Student with the ID ' + @student_id + ' has more than 110 total credits'
        FETCH NEXT FROM above_110_cursor INTO @student_id
    END;
GO

CLOSE above_110_cursor;
DEALLOCATE above_110_cursor;


Commands completed successfully.

Student with the ID 10204 has more than 110 total credits

Student with the ID 10454 has more than 110 total credits

Student with the ID 10527 has more than 110 total credits

Student with the ID 10693 has more than 110 total credits

...

Student with the ID 99710 has more than 110 total credits

Student with the ID 99775 has more than 110 total credits

Student with the ID 99949 has more than 110 total credits

Commands completed successfully.

Total execution time: 00:00:00.089

### ``TRY...CATCH``

The TRY CATCH construct allows you to handle exceptions in SQL Server. To use the TRY CATCH construct, you first place a group of Transact-SQL statements that could cause an exception in a BEGIN TRY...END TRY block as follows:

```
BEGIN TRY  
   -- statements that may cause exceptions
END TRY  
```
Then you use a BEGIN CATCH...END CATCH block immediately after the TRY block:
```
BEGIN CATCH  
   -- statements that handle exception
END CATCH  
```
We have an example below. We define a stored procedure which divindes two values with exception handling.

In [6]:
CREATE PROC usp_divide (
    @a DECIMAL,
    @b DECIMAL,
    @c DECIMAL OUTPUT
) AS
BEGIN
    BEGIN TRY
        SET @c = @a/@b
    END TRY
    BEGIN CATCH
        SELECT 
            ERROR_NUMBER() AS ErrorNumber,
            ERROR_SEVERITY() AS ErrorSeverity,
            ERROR_STATE() AS ErrorState,
            ERROR_PROCEDURE() AS ErrorProcedure,
            ERROR_LINE() AS ErrorLine,
            ERROR_MESSAGE() AS ErrorMessage;  
    END CATCH
END;
GO

Commands completed successfully.

Total execution time: 00:00:00.006

In [12]:
DECLARE @r DECIMAL;
EXEC usp_divide 10, 2, @r OUTPUT;
PRINT @r;

5

Total execution time: 00:00:00.001

Next we run it in a way which will trigger an error. Inside the <code>CATCH</code> block, you can use the following functions to get the detailed information on the error that occurred:

- `ERROR_LINE()` returns the line number on which the exception occurred.
- `ERROR_MESSAGE()` returns the complete text of the generated error message.
- `ERROR_PROCEDURE()` returns the name of the stored procedure or trigger where the error occurred.
- `ERROR_NUMBER()` returns the number of the error that occurred.
- `ERROR_SEVERITY()` returns the severity level of the error that occurred.
- `ERROR_STATE()` returns the state number of the error that occurred.

Note that you only use these functions in the <code>CATCH</code> block. If you use them outside of the <code>CATCH</code> block, all of these functions will return <code>NULL</code>.

In [14]:
DECLARE @r DECIMAL;
EXEC usp_divide 10, 0, @r OUTPUT;
PRINT @r;

(1 row affected)

Total execution time: 00:00:00.008

ErrorNumber,ErrorSeverity,ErrorState,ErrorProcedure,ErrorLine,ErrorMessage
8134,16,1,usp_divide,8,Divide by zero error encountered.


Inside the <code>CATCH</code> block, you can test the state of transactions by using the <code>XACT_STATE()</code> function:

1. If the `XACT_STATE()` function returns -1, it means that an uncommitable transaction is pending, you should issue a rollback transaction statement
2. In case the `XACT_STATE()` function returns 1, it mean that a commitable transaction is pending. You can issue a COMMIT TRANSACTION statement in this case.
3. If the `XACT_STATE()` function returns 0, it mean no transaction is pending therefore you do not neet to take any action.

It is in general a good practice to test our changes to the database before commiting the underlying transaction. In the case below, we will test just that. We define a stored procedure for updating students credits data, but we wrap it around the try catch.

In [3]:
---- Preparing the test data
USE uni;
GO
-- create a dummy table
IF NOT EXISTS(SELECT * FROM sys.objects WHERE [name] = 'student_copy')
    BEGIN
        SELECT * 
        INTO student_copy
        FROM student;
        -- add a check constraint for inducing errors
        ALTER TABLE student_copy
        ADD CONSTRAINT check_total_credits CHECK (tot_cred < 150);
    END
GO
--- Stored procedure 1: reports an error
-- existing, drop it
IF EXISTS (SELECT  * FROM dbo.sysobjects WHERE id = OBJECT_ID(N'[dbo].[sp_report_error]'))
    BEGIN
        DROP PROCEDURE dbo.sp_report_error
    END;
GO
-- define the stored procedure for reporting an error
CREATE PROC sp_report_error
AS
    SELECT   
        ERROR_NUMBER() AS ErrorNumber,
        ERROR_SEVERITY() AS ErrorSeverity, 
        ERROR_STATE() AS ErrorState,
        ERROR_LINE () AS ErrorLine,
        ERROR_PROCEDURE() AS ErrorProcedure, 
        ERROR_MESSAGE() AS ErrorMessage;  
GO

--- Stored Procedure 2: updates total credits of a student with error handling and transaction control
-- existing, drop it
IF EXISTS (SELECT  * FROM dbo.sysobjects WHERE id = OBJECT_ID(N'[dbo].[sp_update_tot_cred]'))
    BEGIN
        DROP PROCEDURE dbo.sp_update_tot_cred
    END;
GO
-- define the stored procedure for updating total credits
CREATE PROC dbo.sp_update_tot_cred (
    @student_id INT,
    @cred_update INT
) AS
BEGIN
    BEGIN TRY
        BEGIN TRANSACTION;
        -- update the grades
        UPDATE student_copy
        SET tot_cred = tot_cred + @cred_update
        WHERE ID = @student_id;
        -- if DELETE succeeds, commit the transaction
        COMMIT TRANSACTION;  
    END TRY
    BEGIN CATCH
            EXEC sp_report_error;
            -- Test if the transaction is uncommittable.  
            IF (XACT_STATE()) = -1  
            BEGIN  
                PRINT  N'The transaction is in an uncommittable state.' +  
                        'Rolling back transaction.'  
                ROLLBACK TRANSACTION;  
            END;  
            -- Test if the transaction is committable.  
            IF (XACT_STATE()) = 1  
            BEGIN  
                PRINT N'The transaction is committable.' +  
                    'Committing transaction.'  
                COMMIT TRANSACTION;     
            END;          
    END CATCH
END;
GO

--- try it
--- Case without error
-- before
SELECT * 
FROM student_copy
WHERE ID = 10033;
go
-- all good...
EXEC dbo.sp_update_tot_cred 
    @student_id = 10033,
    @cred_update = 3;
GO
-- after 
SELECT * 
FROM student_copy
WHERE ID = 10033;
GO
--- case with error
-- before
SELECT * 
FROM student_copy
WHERE ID = 10033;
GO
-- error...
EXEC dbo.sp_update_tot_cred 
    @student_id = 10033,
    @cred_update = 300;
GO
-- after
SELECT * 
FROM student_copy
WHERE ID = 10033;
GO


Commands completed successfully.

(2000 rows affected)

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

...

ID,name,dept_name,tot_cred
10033,Zelty,Mech. Eng.,60


ID,name,dept_name,tot_cred
10033,Zelty,Mech. Eng.,63


ID,name,dept_name,tot_cred
10033,Zelty,Mech. Eng.,63


ErrorNumber,ErrorSeverity,ErrorState,ErrorLine,ErrorProcedure,ErrorMessage
547,16,0,10,dbo.sp_update_tot_cred,"The UPDATE statement conflicted with the CHECK constraint ""check_total_credits"". The conflict occurred in database ""uni"", table ""dbo.student_copy"", column 'tot_cred'."


ID,name,dept_name,tot_cred
10033,Zelty,Mech. Eng.,63


### ``RAISEERROR``

The RAISERROR statement allows you to generate your own error messages and return these messages back to the application using the same format as a system error or warning message generated by SQL Server Database Engine. In addition, the RAISERROR statement allows you to set a specific message id, level of severity, and state for the error messages.

Syntax:

```
RAISERROR ( { message_id | message_text | @local_variable }  
    { ,severity ,state }  
    [ ,argument [ ,...n ] ] )  
    [ WITH option [ ,...n ] ];
```
- ``message_id`` is a user-defined error message number stored in the ``sys.messages`` catalog view. User defined error message numbers should be greater than 50,000.

In [7]:
-- The following code adds a user defined error message to the system catalog
EXEC sp_addmessage 
    @msgnum = 51111, 
    @severity = 1, 
    @msgtext = 'BAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAH!';

--try
RAISERROR (51111, 1, 1)

-- remove it 
EXEC sp_dropmessage 
    @msgnum = 51111;


Msg 51111, Level 1, State 1, Line 8
BAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAH!

Total execution time: 00:00:00.007

* ``message_text`` is a user-defined message which is defined directly in the raiserror funtion and using its default message id (50000).

In [8]:
RAISERROR ('BAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAH!', 1, 1)

Msg 50000, Level 1, State 1, Line 1
BAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAH!

Total execution time: 00:00:00

- ``severity`` level is an integer between 0 and 25 with each level representing the seriousness of the error

```
0–10 Informational messages
11–18 Errors
19–25 Fatal errors
```

* ``WITH OPTION`` allows
    - ``WITH LOG`` logs the error in the error log and application log for the instance of the SQL Server Database Engine.
    - ``WITH NOWAIT`` sends the error message to the client immediately.
    - ``WITH SETERROR`` sets the ERROR_NUMBER and @@ERROR values to message_id or 50000, regardless of the severity level.

We can take the previous SP and add a custom error to be triggered immeadiatly if the input of credits is higher than the limited allowed.

In [15]:
--- Stored Procedure 3: updates total credits of a student with error handling and transaction control
-- existing, drop it
IF EXISTS (SELECT  * FROM dbo.sysobjects WHERE id = OBJECT_ID(N'[dbo].[sp_update_tot_cred]'))
    BEGIN
        DROP PROCEDURE dbo.sp_update_tot_cred
    END;
GO
-- define the stored procedure for updating total credits
CREATE PROC dbo.sp_update_tot_cred (
    @student_id INT,
    @cred_update INT
) AS
BEGIN
    -- raise error if update plus current credits would lead to more than 300
    DECLARE @current_tot_cred INT;
    SELECT @current_tot_cred = tot_cred 
    FROM student_copy
    WHERE ID = @student_id;
    -- if the sum of the two exceeds 300, abort the procedure
    IF (@current_tot_cred + @cred_update) > 300
        -- prep the error custom message
        DECLARE @error_msg VARCHAR(250)
        SET @error_msg = N'A student can only have up to 300 credits'
        DECLARE @excess_creds INT
        SET @excess_creds = STR(@current_tot_cred + @cred_update)
        RAISERROR (
            @error_msg, -- message text
            16, --severity
            1 --state
        )
    BEGIN TRY
        -- check how many the student currently has
        BEGIN TRANSACTION;
        -- update the grades
        UPDATE student_copy
        SET tot_cred = tot_cred + @cred_update
        WHERE ID = @student_id;
        -- if DELETE succeeds, commit the transaction
        COMMIT TRANSACTION;  
    END TRY
    BEGIN CATCH
            EXEC sp_report_error;
            -- Test if the transaction is uncommittable.  
            IF (XACT_STATE()) = -1  
            BEGIN  
                PRINT  N'The transaction is in an uncommittable state.' +  
                        'Rolling back transaction.'  
                ROLLBACK TRANSACTION;  
            END;  
            -- Test if the transaction is committable.  
            IF (XACT_STATE()) = 1  
            BEGIN  
                PRINT N'The transaction is committable.' +  
                    'Committing transaction.'  
                COMMIT TRANSACTION;     
            END;          
    END CATCH
END;
GO

--- try it
-- error...
EXEC dbo.sp_update_tot_cred 
    @student_id = 10033,
    @cred_update = 300;
GO

Commands completed successfully.

Commands completed successfully.

(0 rows affected)

(1 row affected)

The transaction is committable.Committing transaction.

Total execution time: 00:00:00.014

ErrorNumber,ErrorSeverity,ErrorState,ErrorLine,ErrorProcedure,ErrorMessage
547,16,0,28,dbo.sp_update_tot_cred,"The UPDATE statement conflicted with the CHECK constraint ""check_total_credits"". The conflict occurred in database ""uni"", table ""dbo.student_copy"", column 'tot_cred'."


: Msg 50000, Level 16, State 1, Procedure dbo.sp_update_tot_cred, Line 19
A student can only have up to 300 credits

## Dynamic SQL

(Mostly drawing on <a href="https://www.sqlservertutorial.net/sql-server-stored-procedures/sql-server-dynamic-sql/" data-href="https://www.sqlservertutorial.net/sql-server-stored-procedures/sql-server-dynamic-sql/" title="https://www.sqlservertutorial.net/sql-server-stored-procedures/sql-server-dynamic-sql/">this page</a>)

Dynamic SQL is a programming technique that allows you to construct SQL statements dynamically at runtime. It allows you to create more general purpose and flexible SQL statement because the full text of the SQL statements may be unknown at compilation. For example, you can use the dynamic SQL to create a stored procedure that queries data against a table whose name is not known until runtime.

This particularly useful for working with SQL from other programs such as Python or R.

For creating dynamic SQL you just need to turn an SQL expression into a string, e.g.

```
'SELECT TOP 6 * FROM department;'
```

and execute it using the stored procedure <code>sp_executesql</code>.

In [17]:
EXEC sp_executesql N'SELECT TOP 6 * FROM department;';
GO

(6 rows affected)

Total execution time: 00:00:00.003

dept_name,building,budget
Accounting,Saucon,3534727.36
Astronomy,Taylor,4938031.52
Athletics,Bronfman,5876405.60
Biology,Candlestick,5180884.40
Civil Eng.,Chandler,2040331.68
Comp. Sci.,Lamberton,851029.52


Because the <code>sp_executesql</code> accepts the dynamic SQL as a Unicode string, you need to prefix it with an <code>N</code>.

### Using dynamic SQL to query from any table example

1. First, declare two variables, `@table` for holding the name of the table from which you want to query and `@sql` for holding the dynamic SQL.
2. Second, set the value of the `@table` variable to the relevant table.
3. Third, construct the dynamic SQL by concatenating the `SELECT` statement with the table name parameter
4. Fourth, call the `sp_executesql` stored procedure by passing the `@sql` parameter.

In [24]:
BEGIN
    DECLARE @table NVARCHAR(128), @sql NVARCHAR(MAX);
    SET @table = N'department';
    SET @sql = N'SELECT TOP 6 * FROM ' + @table;
    EXEC sp_executesql @sql;
END;
GO

(6 rows affected)

Total execution time: 00:00:00.006

dept_name,building,budget
Accounting,Saucon,3534727.36
Astronomy,Taylor,4938031.52
Athletics,Bronfman,5876405.60
Biology,Candlestick,5180884.40
Civil Eng.,Chandler,2040331.68
Comp. Sci.,Lamberton,851029.52


Now we can define our own stored procedure so as to get the first 6 rows of any table. But lets try to make something more useful. This stored procedure takes as input a database name and a table, it outputs a description of the columns and their data-types.

In [29]:
--- Stored Procedure: Describes columns and the respective data types for a given table in a given database in the server
-- existing, drop it
IF EXISTS (SELECT  * FROM dbo.sysobjects WHERE id = OBJECT_ID(N'[usp_describe_table]'))
    BEGIN
        DROP PROCEDURE usp_describe_table
    END;
GO


CREATE PROC usp_describe_table (
    @table NVARCHAR(128),
    @db NVARCHAR(128) = 'uni' -- default
) AS 
BEGIN
    -- declare SQL 
    DECLARE @sql NVARCHAR(MAX)
    -- construct the SQL
    SET @sql = N'USE ' + @db + N'; 
    SELECT 
    c.name [Column Name],
    t.Name [Data type],
    c.max_length [Max Length],
    c.precision ,
    c.scale ,
    c.is_nullable,
    ISNULL(i.is_primary_key, 0) [Primary Key]
FROM    
    sys.columns c
INNER JOIN 
    sys.types t ON c.user_type_id = t.user_type_id
LEFT OUTER JOIN 
    sys.index_columns ic ON ic.object_id = c.object_id AND ic.column_id = c.column_id
LEFT OUTER JOIN 
    sys.indexes i ON ic.object_id = i.object_id AND ic.index_id = i.index_id
WHERE
    c.object_id = OBJECT_ID(''' + @table +''');'
    -- execute it
    EXEC sp_executesql @sql;
END;
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.008

In [32]:
-- test it
EXEC usp_describe_table @db = 'uni', @table = 'takes';
GO
-- test it with another database
EXEC usp_describe_table @db = 'dbm_project', @table = 'RETURNS';
GO

(6 rows affected)

(7 rows affected)

Total execution time: 00:00:00.050

Column Name,Data type,Max Length,precision,scale,is_nullable,Primary Key
ID,varchar,5,0,0,0,1
course_id,varchar,8,0,0,0,1
sec_id,varchar,8,0,0,0,1
semester,varchar,6,0,0,0,1
year,numeric,5,4,0,0,1
grade,varchar,2,0,0,1,0


Column Name,Data type,Max Length,precision,scale,is_nullable,Primary Key
IDRETURN,numeric,9,18,0,0,1
PRODUCT,numeric,9,18,0,0,0
WAREHOUSE,numeric,9,18,0,0,0
SALE,numeric,9,18,0,0,0
RETURN_DATE,datetime,8,23,3,0,0
RETURN_QUANTITY,smallint,2,5,0,0,0
RETURN_REASON,varchar,100,0,0,0,0


### Detour: SQL injections

Take this simple stored procedure which queries a table.

In [35]:
--- Stored Procedure: Queries all rows and cols in a table
-- existing, drop it
IF EXISTS (SELECT * FROM dbo.sysobjects WHERE id = OBJECT_ID(N'[usp_simple_query]'))
    BEGIN
        DROP PROCEDURE usp_simple_query
    END;
GO
CREATE PROC usp_simple_query (
    @table NVARCHAR(128)
) AS 
BEGIN
    -- declare SQL 
    DECLARE @sql NVARCHAR(MAX)
    -- construct the SQL
    SET @sql = N'SELECT TOP 6 * FROM ' + @table + ';'
    -- execute it
    EXEC sp_executesql @sql;
END;
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.009

While only taking as input the table name, this statement does not prevent users from passing more SQL commands in the table name parameter. This unknowingly allowing database users to possibly run SQL statements which alter or retrieve data from a dataase is called ``SQL injection``. For example, here the SQL injection will lead to the destruction of a table by adding another SQL command after the query.

In [42]:
-- create a dummy table
IF NOT EXISTS(SELECT * FROM sys.objects WHERE [name] = 'student_copy')
    BEGIN
        SELECT * 
        INTO student_copy
        FROM student;
    END
GO

-- before the injection
SELECT TABLE_NAME
FROM INFORMATION_SCHEMA.TABLES
WHERE TABLE_NAME = 'student_copy';
GO
-- SQL injection
EXEC usp_simple_query @table = 'student_copy; DROP TABLE student_copy;'

-- after the injection
SELECT TABLE_NAME
FROM INFORMATION_SCHEMA.TABLES
WHERE TABLE_NAME = 'student_copy';
GO

(2000 rows affected)

(1 row affected)

(6 rows affected)

(0 rows affected)

Total execution time: 00:00:00.030

TABLE_NAME
student_copy


ID,name,dept_name,tot_cred
1000,Manber,Civil Eng.,39
10033,Zelty,Mech. Eng.,60
10076,Duan,Civil Eng.,105
1018,Colin,Civil Eng.,81
10204,Mediratta,Geology,112
10267,Rzecz,Comp. Sci.,5


TABLE_NAME


In this example, closer to issues related with data security, we get acess to the students data even if we do not have a student id. The main takeaway is that the where clause will filter the amount of data returned based on evaluating the logical predicates present. If we add an optional predicate which always evaluate to ``TRUE`` we can retrieve all the data as it will evaluate as true for the entire table.

In [87]:
--- Stored Procedure: Queries all rows and cols in a table
-- existing, drop it
IF EXISTS (SELECT * FROM dbo.sysobjects WHERE id = OBJECT_ID(N'[usp_get_student]'))
    BEGIN
        DROP PROCEDURE usp_get_student
    END;
GO
CREATE PROC usp_get_student (
    @student_name VARCHAR(200)
) AS 
BEGIN
    -- declare SQL 
    DECLARE @sql NVARCHAR(MAX)
    -- construct the SQL
    SET @sql = N'SELECT TOP 6 * FROM student WHERE name = ' + @student_name
    -- execute it
    EXEC sp_executesql @sql;
END;
GO

-- normal query with match
EXEC usp_get_student @student_name = '''Zelty''';
GO
-- normal query without match
EXEC usp_get_student @student_name = '''Mickey Mouse''';
GO
-- injection of fake name plus a logical condition which evaluates to TRUE
EXEC usp_get_student @student_name = ' ''Mickey Mouse'' OR 1 + 1 = 2 ';
GO

Commands completed successfully.

Commands completed successfully.

(1 row affected)

(0 rows affected)

(6 rows affected)

Total execution time: 00:00:00.017

ID,name,dept_name,tot_cred
10033,Zelty,Mech. Eng.,60


ID,name,dept_name,tot_cred


ID,name,dept_name,tot_cred
1000,Manber,Civil Eng.,39
10033,Zelty,Mech. Eng.,60
10076,Duan,Civil Eng.,105
1018,Colin,Civil Eng.,81
10204,Mediratta,Geology,112
10267,Rzecz,Comp. Sci.,5


Below you have one possible way of curbing this issue, though there are more such as paramaterizing the query etc.

In [84]:
--- Stored Procedure: Queries all rows and cols in a table
-- existing, drop it
IF EXISTS (SELECT * FROM dbo.sysobjects WHERE id = OBJECT_ID(N'[usp_safely_get_student]'))
    BEGIN
        DROP PROCEDURE usp_safely_get_student
    END;
GO
CREATE PROC usp_safely_get_student (
    @student_name VARCHAR(200)
) AS 
BEGIN
    -- declare SQL 
    DECLARE @sql NVARCHAR(MAX)
    -- construct the SQL
    SET @sql = N'SELECT TOP 6 * FROM student WHERE name = ''' + REPLACE(@student_name,'''', '''''') + ''''
    -- execute it
    EXEC sp_executesql @sql;
END;
GO

-- normal query with match
EXEC usp_safely_get_student @student_name = 'Zelty';
GO
-- normal query without match
EXEC usp_safely_get_student @student_name = 'Mickey Mouse';
GO
-- injection of fake name plus a logical condition which evaluates to TRUE
EXEC usp_safely_get_student @student_name = ' ''Mickey Mouse'' OR 1 + 1 = 2 ';
GO

Commands completed successfully.

Commands completed successfully.

(1 row affected)

(0 rows affected)

(0 rows affected)

Total execution time: 00:00:00.019

ID,name,dept_name,tot_cred
10033,Zelty,Mech. Eng.,60


ID,name,dept_name,tot_cred


ID,name,dept_name,tot_cred
